In [37]:
import optuna
from optuna.integration import TFKerasPruningCallback
from optuna.trial import TrialState

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
import logging
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
import math

logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

datapath = 'data\\reduced_temperature_data.xlsx'

X_data = np.array(pd.read_excel(datapath, sheet_name = 'boundary_condition'))
Y_data = np.transpose(np.array(pd.read_excel(datapath, sheet_name = 'temperature')))

(45, 3)
(45, 4096)


In [38]:
from optuna import delete_study
from optuna.exceptions import StorageInternalError
import time
from tensorflow.keras import layers
start =time.time()

training_condition = (X_data[:,1] == 10)|(X_data[:,1] == 12.5)|(X_data[:,1] == 15)|(X_data[:,1] == 17.5)
training_row = np.where(training_condition)[0]

validation_condition = (X_data[:,1] == 20)
validation_row = np.where(validation_condition)[0]

#train + validation
x_train_data = X_data[training_row,:]
y_train_data = Y_data[training_row,:]
x_val_data = X_data[validation_row,:]
y_val_data = Y_data[validation_row,:]

#standardization
x_train_scaled = (x_train_data - np.mean(x_train_data))/np.std(x_train_data)
y_train_scaled = (y_train_data - np.mean(y_train_data))/np.std(y_train_data)
x_val_scaled = (x_val_data - np.mean(x_train_data))/np.std(x_train_data)
y_val_scaled = (y_val_data - np.mean(y_train_data))/np.std(y_train_data)

tf.random.set_seed(1234)  # applied to achieve consistent results

#early_stop settings
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

def create_model(trial):
    n_layers = trial.suggest_int('n_layers', 1, 5)
    model = tf.keras.models.Sequential()
    for i in range(n_layers):
        num_hidden = trial.suggest_int('n_units_{}'.format(i), 4, 64, log=True)
        model.add(layers.Dense(num_hidden, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01)))
        dropout = trial.suggest_float('dropout_{}'.format(i), 0.1, 0.5)
        model.add(layers.Dropout(rate=dropout))
    model.add(layers.Dense(np.shape(Y_data)[1]))
    
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)       
    model.compile(loss='mae',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  metrics=['mae', 'mse'])

    return model

def objective(trial):
    
    tf.keras.backend.clear_session()

    x_train = x_train_scaled
    x_valid = x_val_scaled
    y_train = y_train_scaled
    y_valid = y_val_scaled  

    model = create_model(trial)

    model.fit(x_train, y_train,
              callbacks=early_stop,
              epochs=2000,
              validation_data=(x_valid, y_valid),
              verbose=0)
    def r_squared(y_true, y_pred):
        mean = np.mean(y_true)
        total_sum_squares = np.sum((y_true - mean) ** 2)
        residual_sum_squares = np.sum((y_true - y_pred) ** 2)
        r2 = 1 - (residual_sum_squares / total_sum_squares)
        return r2

    score = r_squared(y_valid, model.predict(x_valid,verbose=0))
    return score

#optimization
if __name__ == '__main__':
    storage_name = 'sqlite:///CFD_ANN_opt.db'
    
    study_name='CFD_ANN_opt'
    try:
        delete_study(storage=storage_name, study_name=study_name)
    except StorageInternalError:
        pass
    
    study = optuna.create_study(
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=10), direction='maximize',
        study_name=study_name, storage=storage_name, load_if_exists=False
    )
    study.optimize(objective, n_trials=20)
    
    best_params = study.best_params
    best_value = study.best_value
    print('\n\nbest_value = '+str(best_value))
    print('best_params:')
    print(best_params)

end=time.time()
print('Running time: %s Seconds'%(end-start))

[I 2024-03-17 18:17:17,048] A new study created in RDB with name: CFD_ANN_opt
[I 2024-03-17 18:17:19,326] Trial 0 finished with value: -0.023641930733292682 and parameters: {'n_layers': 5, 'n_units_0': 11, 'dropout_0': 0.4270865407692739, 'n_units_1': 7, 'dropout_1': 0.46333991010010234, 'n_units_2': 4, 'dropout_2': 0.19871889584609315, 'n_units_3': 39, 'dropout_3': 0.48544527568610174, 'n_units_4': 23, 'dropout_4': 0.37710155101970655, 'learning_rate': 0.0017789376943571546}. Best is trial 0 with value: -0.023641930733292682.
[I 2024-03-17 18:18:07,914] Trial 1 finished with value: 0.6417287510882586 and parameters: {'n_layers': 5, 'n_units_0': 18, 'dropout_0': 0.47170642671550034, 'n_units_1': 14, 'dropout_1': 0.12973311098619122, 'n_units_2': 19, 'dropout_2': 0.13033129020211215, 'n_units_3': 62, 'dropout_3': 0.18397149620032477, 'n_units_4': 40, 'dropout_4': 0.2877075848857696, 'learning_rate': 0.00015336317499991815}. Best is trial 1 with value: 0.6417287510882586.
[I 2024-03-17 1



best_value = 0.9787103471426636
best_params:
{'n_layers': 1, 'n_units_0': 29, 'dropout_0': 0.2750119831757307, 'learning_rate': 0.027988326072001492}
Running time: 136.12247276306152 Seconds


In [36]:
start =time.time()

from tensorflow.keras.layers import Dropout

def create_model(best_params):
    model = Sequential()
    for i in range(best_params['n_layers']):
        model.add(Dense(best_params[f'n_units_{i}'], activation='relu'))
        model.add(Dropout(best_params[f'dropout_{i}']))
    model.add(Dense(np.shape(Y_data)[1]))
    optimizer = tf.keras.optimizers.Adam(learning_rate=best_params['learning_rate'])
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    return model

test_condition = (X_data[:,1] == 12.5)|(X_data[:,1] == 17.5)|(X_data[:,1] == 22.5)|(X_data[:,1] == 27.5)
test_row = np.where(test_condition)[0]

x_test = (X_data[test_row,:] - np.mean(x_train_data))/np.std(x_train_data)
y_test = Y_data[test_row,:]

x_train = x_train_scaled
y_train = y_train_scaled

#model training
model = create_model(best_params)
model.fit(x_train, y_train, epochs=2000, verbose=0)

y_pred = model.predict(x_test)*np.std(y_train_data) + np.mean(y_train_data)

NRMSE = []
for i in range(x_test.shape[0]):
    NRMSE.append((mean_squared_error(y_test[i,:], y_pred[i,:]))**0.5/(np.max(y_test[i,:])-np.min(y_test[i,:])))

MAE = []
for i in range(x_test.shape[0]):
    MAE.append(mean_absolute_error(y_test[i,:], y_pred[i,:]))

model.summary()
print(np.mean(MAE),np.std(MAE))
print(np.round(MAE,2))
print(np.mean(NRMSE))
print(np.round(NRMSE,2))

end=time.time()

df = pd.DataFrame(y_pred)
df.to_excel('ANN.xlsx', index=False)
print('Running time: %s Seconds'%(end-start))

1/1 [==============================] - 0s 45ms/step
Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 38)                152       
                                                                 
 dropout_16 (Dropout)        (None, 38)                0         
                                                                 
 dense_31 (Dense)            (None, 20)                780       
                                                                 
 dropout_17 (Dropout)        (None, 20)                0         
                                                                 
 dense_32 (Dense)            (None, 62)                1302      
                                                                 
 dropout_18 (Dropout)        (None, 62)                0         
                                                                 
 